# Convert Naryn monthly flow rate data to monthly volumes.
Assumes the flow rate is an average that can be multiplied by days in the month for 
total monthly volume.

In [ ]:
from __future__ import print_function
import calendar
from charistools.hypsometry import Hypsometry
from charistools.timeSeries import TimeSeries
import pandas as pd
import numpy as np
import os
import re
from time import strptime

In [ ]:
%cd /Users/brodzik/projects/CHARIS/streamflow/From_Alice_Naryn
%ls

In [ ]:
file = "Kyrygzstan_Naryn_flow_monthly_average_1933-2015.xlsx"

In [ ]:
# Use sheetname if pandas < 0.21.0
# use sheet_name if later
# Returns dict of sheets in file
sheets = pd.read_excel(io=file, sheetname=None)

In [ ]:
for key in sheets:
    print("key=%s" % key)

In [ ]:
# Set the source name from the original file (controls which sheet is extracted)
# and the CHARIS-format drainageID
sheetName = "Naryn town gauge (Naryn Darya)"
drainageID = "SY_Naryn_at_NarynTown"

In [ ]:
df = sheets[sheetName]
df

In [ ]:
description = df.columns[0]
description

In [ ]:
geolocation = df.iloc[0][0]
geolocation

In [ ]:
units = df.iloc[1][0]
units

In [ ]:
description, geolocation, units

In [ ]:
def dmsds_to_decdeg(deg, min, sec, decsec):
    return float(deg) + float(min)/60. + (float(sec) + float(decsec)/100.)/60./60.

In [ ]:
# Convert lat/lon positions to decimal degrees
p = re.compile(r'latitude:(.*)longitude:(.*)\(')
m = p.search(geolocation)
if m :
    print(m.group(1))
    print(m.group(2))
    
    (deg, min, sec, decsec) = m.group(1).split()
    latitude = dmsds_to_decdeg(deg, min, sec, decsec)
    (deg, min, sec, decsec) = m.group(2).split()
    longitude = dmsds_to_decdeg(deg, min, sec, decsec)
    print("lat/lon: %s/%s" % (latitude, longitude))
    
else:
    print("no match")


In [ ]:
# Drop the first 2 data rows, they are just metadata we have already scraped
df = df.drop([0,1])

In [ ]:
#sheets['Naryn town gauge (Naryn Darya)']
df.loc[2].values

In [ ]:
# Move the contents of data row 2 into the column headers
df.columns = df.loc[2].values
df

In [ ]:
# Now drop the data row 2
df = df.drop([2])
df

In [ ]:
# Move the df index to the Year column
df = df.set_index('Year')
df

In [ ]:
df['Jan'][1933]

In [ ]:
# Write the result as a hypsometry for ti-melt modeling calibration steps, this needs to look like this:
# # Station ID: 35749301
# # Basin: Hunza at Dainyor Bridge
# # Latitude: 35.92778
# # Longitude: 74.37639
# # Elevation: 1370 m
# # Area: 13157 km**2
# # Units: km**3
# # COLUMNS: year month runoff
# 1966  1      0.15
# 1966  2      0.12
# 1966  3      0.10
outfile = "%s.month_runoff.dat" % drainageID
outfile

In [ ]:
comments=["Description: Derived runoff volume from monthly flow rates",
          "Flow rate file: %s" % os.path.basename(file),
          "Basin: %s" % drainageID,
          "Latitude: %.6f" % latitude,
          "Longitude: %.6f" % longitude,
          "Elevation: unknown",
          "Area: unknown",
          "Units: km^3",
          "COLUMNS: year month runoff"]
f = open(outfile, "w")
for line in comments:
    f.write("# %s\n" % line)

# Convert the monthly average flow rates to volumes using:

$Volume(km^3/month) = \frac{m^3}{s} * \frac{1 km}{1000 m} * \frac{1 km}{1000 m}* \frac{1 km}{1000 m} * \frac{60 * 60 * 24 s}{1 day} * \frac{N days}{month}$

In [ ]:
m_per_km = 1000.
s_per_day = 60. * 60. * 24.
conversion_factor = s_per_day / (m_per_km * m_per_km * m_per_km)
for year in df.index:
    for mmm in df.columns[1:13]:
        month = strptime(mmm,'%b').tm_mon
        days_in_month = calendar.monthrange(year, int(month))[1]
        volume_km3 = df[mmm][year] * conversion_factor * days_in_month
        print("%4d %d %d %f %f" % (year, month, days_in_month, df[mmm][year], volume_km3))
        print("%4d %2d %.6f" % (year, month, volume_km3),
              file=f)

In [ ]:
f.close()

In [ ]:
%
%ls *month_runoff*


In [ ]:
%pwd
ts = TimeSeries("../SY_Naryn_at_NarynTown.month_runoff.dat")

In [ ]:
ts.data